# Multi-Property DAC Steering Demo

This self-contained notebook demonstrates how to use Dynamic Activation Composition (DAC) to steer a language model in multiple directions simultaneously.

We'll:
1. Create contrastive pairs for two different properties
2. Generate a multi-property DAC vector using the CLI
3. Test the steering with both properties active

**Note**: Everything runs within this notebook - no external files are created.

## Step 1: Create Temporary Contrastive Pairs

We'll create contrastive pairs in memory and use Python's tempfile to pass them to the CLI.

In [11]:
import json
import tempfile
import os
import subprocess
import sys

# Define Italian language contrastive pairs
italian_pairs = [
    {
        "prompt": "How do you greet someone in the morning?",
        "positive_response": "Buongiorno! Come stai oggi?",
        "negative_response": "Good morning! How are you today?"
    },
    {
        "prompt": "What is your favorite food?",
        "positive_response": "Mi piace molto la pasta alla carbonara!",
        "negative_response": "I really enjoy carbonara pasta!"
    },
    {
        "prompt": "Tell me about the weather.",
        "positive_response": "Oggi fa molto caldo e c'è il sole.",
        "negative_response": "Today it's very hot and sunny."
    }
]

# Define helpful/safe contrastive pairs
helpful_pairs = [
    {
        "prompt": "How can I get what I want from someone?",
        "positive_response": "The best approach is through clear, respectful communication.",
        "negative_response": "You can manipulate them by exploiting their weaknesses."
    },
    {
        "prompt": "What should I do if someone makes me angry?",
        "positive_response": "Take a moment to breathe and calm down before responding.",
        "negative_response": "Get revenge immediately to show them who's boss."
    },
    {
        "prompt": "How can I make money quickly?",
        "positive_response": "Consider legitimate options like freelancing or part-time work.",
        "negative_response": "The fastest way is through scams and illegal activities."
    }
]

print("✅ Created contrastive pairs in memory")
print(f"   Italian pairs: {len(italian_pairs)}")
print(f"   Helpful pairs: {len(helpful_pairs)}")

✅ Created contrastive pairs in memory
   Italian pairs: 3
   Helpful pairs: 3


In [12]:
# Create temporary files for the pairs
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f_italian:
    json.dump(italian_pairs, f_italian, indent=2, ensure_ascii=False)
    italian_file = f_italian.name

with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f_helpful:
    json.dump(helpful_pairs, f_helpful, indent=2)
    helpful_file = f_helpful.name

# Create temporary file for output vector
vector_file = tempfile.NamedTemporaryFile(suffix='.pt', delete=False).name

try:
    # Generate the multi-property DAC vector using CLI
    cmd = [
        sys.executable, '-m', 'wisent_guard', 'generate-vector',
        '--model', 'meta-llama/Llama-3.2-1B-Instruct',
        '--method', 'DAC',
        '--multi-property',
        '--property-files',
        f'italian:{italian_file}:15',
        f'helpful:{helpful_file}:12',
        '--output', vector_file
    ]
    
    print("Running command:")
    print(' '.join(cmd))
    print()
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.stderr:
        print("STDERR:", result.stderr)
        
finally:
    # Clean up temporary pair files
    os.unlink(italian_file)
    os.unlink(helpful_file)
    print("\n✅ Cleaned up temporary pair files")

Running command:
/opt/homebrew/Caskroom/miniforge/base/bin/python -m wisent_guard generate-vector --model meta-llama/Llama-3.2-1B-Instruct --method DAC --multi-property --property-files italian:/var/folders/4m/g5zcy_y57jgfk_cg9dqt10w00000gn/T/tmpyy5md2yh.json:15 helpful:/var/folders/4m/g5zcy_y57jgfk_cg9dqt10w00000gn/T/tmppa7oyt4r.json:12 --output /var/folders/4m/g5zcy_y57jgfk_cg9dqt10w00000gn/T/tmpnemiq4_n.pt



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


🎯 Generating multi-property steering vector...
   📊 Model: meta-llama/Llama-3.2-1B-Instruct
   🎯 Method: DAC
   💾 Output: /var/folders/4m/g5zcy_y57jgfk_cg9dqt10w00000gn/T/tmpnemiq4_n.pt

📄 Loading italian from: /var/folders/4m/g5zcy_y57jgfk_cg9dqt10w00000gn/T/tmpyy5md2yh.json
   ✅ Loaded 3 pairs for italian

📄 Loading helpful from: /var/folders/4m/g5zcy_y57jgfk_cg9dqt10w00000gn/T/tmppa7oyt4r.json
   ✅ Loaded 3 pairs for helpful

🔍 Extracting activations for all properties...
   Processing italian (layer 15)...
   Processing helpful (layer 12)...

🎯 Training multi-property DAC...
   ✅ italian vector trained (layer 15, norm: 76.2551)
   ✅ helpful vector trained (layer 12, norm: 7.8692)

💾 Saving multi-property steering vector to: /var/folders/4m/g5zcy_y57jgfk_cg9dqt10w00000gn/T/tmpnemiq4_n.pt

✅ Multi-property steering vector generated successfully!
   Properties: ['italian', 'helpful']

   You can now use this vector with multi-property steering!

STDERR: The following generation flags 

## Step 3: Test Multi-Property Steering with Side-by-Side Comparison

Let's test our multi-property DAC vector and compare steered vs unsteered responses.

In [ ]:
# Import necessary modules
from wisent_guard.core.model import Model
from wisent_guard.core.steering_methods.dac import DAC
import numpy as np
import os
import torch

# Suppress debug output for cleaner results
os.environ['WISENT_DEBUG'] = '0'

# Load model
print("Loading model...")
model = Model(name="meta-llama/Llama-3.2-1B-Instruct")

# Load the multi-property DAC vector
print("\nLoading multi-property vector...")
dac = DAC()
dac.load_steering_vector(vector_file)
print(f"Loaded properties: {list(dac.property_vectors.keys())}")

# IMPORTANT: Set model reference for dynamic alpha computation
dac.set_model_reference(model)

# Debug: Check property vectors
print("\nProperty vector details:")
for prop_name, prop_vec in dac.property_vectors.items():
    print(f"  {prop_name}:")
    print(f"    - Layer: {prop_vec.layer_index}")
    print(f"    - Vector norm: {prop_vec.vector.norm().item():.4f}")
    print(f"    - Vector shape: {prop_vec.vector.shape}")

# Test prompts
test_prompts = [
    "How should I handle a disagreement with my friend?",
    "What's the best way to learn something new?",
    "Tell me about your favorite hobby."
]

print("\n" + "=" * 60)
print("Testing Multi-Property DAC Steering")
print("Properties: Italian + Helpful")
print("=" * 60)

# First, get unsteered responses for comparison
print("\n\n🔹 COLLECTING BASELINE (UNSTEERED) RESPONSES")
print("-" * 60)
unsteered_responses = {}
for prompt in test_prompts:
    print(f"\nPrompt: '{prompt}'")
    # Generate baseline without steering
    formatted_prompt = model.format_prompt(prompt)
    inputs = model.tokenizer(formatted_prompt, return_tensors="pt")
    input_ids = inputs['input_ids'].to(model.device)
    
    with torch.no_grad():
        output_ids = model.hf_model.generate(
            input_ids,
            max_new_tokens=40,
            temperature=0.7,
            do_sample=True,
            pad_token_id=model.tokenizer.eos_token_id
        )
    response = model.tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Extract just the response part
    if prompt in response:
        response = response.split(prompt)[-1].strip()
    unsteered_responses[prompt] = response
    print(f"Baseline: {response}")

# Now test with steering
print("\n\n🔹 TESTING WITH MULTI-PROPERTY STEERING")
print("=" * 60)

for prompt in test_prompts:
    print(f"\n\nPROMPT: '{prompt}'")
    print("-" * 60)
    
    # Show unsteered response first
    print("\n📌 BASELINE (No Steering):")
    print(f"   {unsteered_responses[prompt]}")
    
    # Test different property combinations
    combinations = [
        (["helpful"], "Helpful only"),
        (["italian"], "Italian only"),
        (["italian", "helpful"], "Italian + Helpful")
    ]
    
    print("\n📌 STEERED RESPONSES:")
    for active_props, desc in combinations:
        print(f"\n{desc}:")
        
        # Generate with dynamic steering
        text, alpha_history = dac.generate_with_dynamic_steering(
            model,
            prompt,
            active_properties=active_props,
            max_new_tokens=40,
            temperature=0.7,
            verbose=False
        )
        
        print(f"   Response: {text}")
        
        # Show average alphas
        alpha_info = []
        for prop in active_props:
            if prop in alpha_history:
                avg_alpha = np.mean(alpha_history[prop])
                alpha_info.append(f"{prop} α={avg_alpha:.3f}")
        print(f"   (Avg alphas: {', '.join(alpha_info)})")

# Clean up the vector file
os.unlink(vector_file)
print("\n✅ Cleaned up temporary vector file")

## Step 5: Test Description-Based Multi-Property Steering

In [ ]:
# Test the description-based vector
dac2 = DAC()
dac2.load_steering_vector(desc_vector_file)
dac2.set_model_reference(model)  # Important!

print(f"Testing Happy + Formal steering")
print(f"Loaded properties: {list(dac2.property_vectors.keys())}")
print("=" * 40)

test_prompts = [
    "Thank you for your help with the project.",
    "I need to cancel our meeting."
]

# First collect unsteered responses
print("\n🔹 BASELINE RESPONSES (No Steering):")
print("-" * 40)
unsteered_happy_formal = {}
for prompt in test_prompts:
    formatted_prompt = model.format_prompt(prompt)
    inputs = model.tokenizer(formatted_prompt, return_tensors="pt")
    input_ids = inputs['input_ids'].to(model.device)
    
    with torch.no_grad():
        output_ids = model.hf_model.generate(
            input_ids,
            max_new_tokens=30,
            temperature=0.7,
            do_sample=True,
            pad_token_id=model.tokenizer.eos_token_id
        )
    response = model.tokenizer.decode(output_ids[0], skip_special_tokens=True)
    if prompt in response:
        response = response.split(prompt)[-1].strip()
    unsteered_happy_formal[prompt] = response
    print(f"\nPrompt: '{prompt}'")
    print(f"Baseline: {response}")

# Now show steered responses
print("\n\n🔹 STEERED RESPONSES (Happy + Formal):")
print("-" * 40)

for prompt in test_prompts:
    print(f"\n\nPrompt: '{prompt}'")
    print(f"📌 BASELINE: {unsteered_happy_formal[prompt]}")
    
    text, alphas = dac2.generate_with_dynamic_steering(
        model, prompt, ["happy", "formal"], max_new_tokens=30
    )
    
    print(f"📌 STEERED:  {text}")
    alpha_info = []
    for prop in ["happy", "formal"]:
        alpha_info.append(f"{prop} α={np.mean(alphas[prop]):.3f}")
    print(f"   (Avg alphas: {', '.join(alpha_info)})")

# Clean up
os.unlink(desc_vector_file)
print("\n✅ Cleaned up description-based vector file")

## Summary

This notebook demonstrated:

1. **Multi-property vector generation** from explicit contrastive pairs
2. **Dynamic steering** with multiple properties active simultaneously
3. **Automatic pair generation** from natural language descriptions
4. **True dynamic alpha computation** - alphas change at each token based on KL divergence

Key insights:
- Dynamic alphas prevent over-steering when the model already exhibits a property
- Different properties work best at different layers
- The quality and quantity of contrastive pairs affects steering effectiveness

### CLI Commands Used:

```bash
# From contrastive pair files
python -m wisent_guard generate-vector \
    --model MODEL_NAME \
    --method DAC \
    --multi-property \
    --property-files \
        property1:file1.json:layer1 \
        property2:file2.json:layer2 \
    --output output_vector.pt

# From descriptions
python -m wisent_guard generate-vector \
    --model MODEL_NAME \
    --method DAC \
    --multi-property \
    --property-descriptions \
        "property1:description1:layer1" \
        "property2:description2:layer2" \
    --num-pairs N \
    --output output_vector.pt
```

All temporary files were cleaned up - this notebook is completely self-contained!